# Desmos Pastebin
### This notebook shows how to upload arbitary text/data to Desmos Calculator and makes it a pastebin service

In [1]:
import requests

import string
import random
import base64

In [2]:
# Maximum upload body size
MAX_BODY_SIZE = 5*2**20  # 5MiB

# Maximum text size
# 91 is the size of base body
MAX_TEXT_SIZE = MAX_BODY_SIZE - 91

# Maximum data size
# 4/3 is the base64 overhead factor
MAX_DATA_SIZE = int(MAX_TEXT_SIZE / (4/3)) # = 3932091

In [3]:
def built_data(data, graph_hash=None):
    # if `graph_hash` is specified,
    # it can only be a unique string of lowercase letters and ditgits
    
    if isinstance(data, str):
        assert len(data) <= MAX_TEXT_SIZE, f'{len(data)} > {MAX_TEXT_SIZE}'
    elif isinstance(data, bytes):
        assert len(data) <= MAX_DATA_SIZE, f'{len(data)} > {MAX_DATA_SIZE}'
        # use altchars '-' and '.' to avoid size inflation due to url encoding of '+' and '/'
        data = base64.b64encode(data, altchars=b'-.').decode()
    else:
        raise TypeError('data should be either str or bytes type')

    if graph_hash is None:
        graph_hash = ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))

    return {
        'thumb_data': 'data:image/png;base64,',
        'graph_hash': graph_hash,
        'my_graphs': 'false',
        'calc_state': data,
    }

In [4]:
random.seed('fuckyougod')
mydata = random.randbytes(MAX_DATA_SIZE)

# random.seed()  # reset seed for random graph_hash
data = built_data(mydata, 'fuckyougod')
data['graph_hash']

'fuckyougod'

In [5]:
prep = requests.Request('POST', 'https://www.desmos.com/api/v1/calculator/save', data=data).prepare()
assert len(prep.body) <= MAX_BODY_SIZE, len(prep.body)

MAX_BODY_SIZE - len(prep.body)

1

In [6]:
session = requests.Session()
response = session.send(prep)

# response status code of 500 is normal
# 409 if graph_hash is already in use (not unique)
# 413 if request body size is more than 5MiB
response.status_code, response.reason

(500, 'Internal Server Error')

In [15]:
# test uploaded data integrity

response = requests.get('https://www.desmos.com/calc-states/production/fuckyougod')
random.seed('fuckyougod')
assert base64.b64decode(response.text, altchars=b'-.') == random.randbytes(MAX_DATA_SIZE)
assert response.headers['Last-Modified'] == 'Tue, 07 Nov 2023 01:55:27 GMT'

### Note on `thumb_data`
`thumb_data` can be used to upload an arbitary PNG image by appending `data:image/png;base64,` with the base64 of the PNG file.  
Because the image has to be in base64 and can not use altchars, its max size is limited to about 3.53 MiB (5MiB * 12/17).  
The url of the image would be https://www.desmos.com/calc_thumbs/production/fuckyougod.png, where `fuckyougod` is the `graph_hash`.